In [1]:
# Dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

import numpy as np

%matplotlib inline

In [2]:
scale: float = 0.5
batch_size: int = 32

In [3]:
test_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ]
)

# noinspection PyArgumentEqualDefault
train_transform = transforms.Compose(
    [
        transforms.RandomCrop(size=32),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomApply([
                transforms.ColorJitter(brightness=0.8*scale, contrast=0.8*scale, saturation=0.8*scale, hue=0.2*scale)
            ], p=0.2
        ),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ]
)

In [4]:
# noinspection pyArgumentEqualDefault,PyArgumentEqualDefault,PyArgumentEqualDefault
cifar_train = torchvision.datasets.CIFAR10(
    root='./data/cifar10/', train=True, transform=train_transform, download=True
)
cifar_test = torchvision.datasets.CIFAR10(
    root='./data/cifar10/', train=False, transform=test_transform, download=True
)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# noinspection pyArgumentEqualDefault
train_dataloader = data.DataLoader(
    dataset=cifar_train, batch_size=batch_size, num_workers=2, shuffle=True
)
# noinspection pyArgumentEqualDefault,PyArgumentEqualDefault
test_dataloader = data.DataLoader(
    dataset=cifar_test, batch_size=batch_size, num_workers=2, shuffle=False
)

In [6]:
class LeNet(nn.Module):
    r"""Implements the basic LeNet architecture.
    """

    def __init__(self):
        r"""The initializer.

        Parameters
        ----------

        Returns
        -------
        """
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=6, kernel_size=(5, 5)
        )
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=6, out_channels=16, kernel_size=(5, 5)
        )
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2))
        self.fc1 = nn.Linear(in_features=16*5*5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    #
    def forward(self, x):
        r"""Implements the forward pass of the network.

        Parameters
        ----------
        x:
            The input data tensor. SHAPE: [B, C, H, W].

        Returns
        -------
        logits (implicit):
            The logits tensor for the input. SHAPE: [B, 10].
        """
        y1 = self.maxpool1(F.relu(self.conv1(x)))
        y2 = self.maxpool2(F.relu(self.conv2(y1)))
        y3 = F.relu(self.fc1(y2.view(y2.shape[0], -1)))
        y4 = F.relu(self.fc2(y3))
        return self.fc3(y4)


In [7]:
model = LeNet()

In [8]:
optimizer = optim.SGD(params=model.parameters(), lr=1e-3, momentum=0.9)

In [14]:
epochs: int = 2

for epoch in range(epochs):

    total_loss: float = 0.0

    model = model.train()

    # For each batch, ...
    for batch_idx, (imgs, labels) in enumerate(iterable=train_dataloader, start=0):

        if batch_idx%500 == 0 or batch_idx == len(train_dataloader) - 1:
            print('[TRAINING] Training batch: {}/{}'.format(batch_idx, int(np.ceil(len(train_dataloader)))))

        logits = model(imgs)
        loss = F.cross_entropy(input=logits, target=labels, reduction='sum')

        # Clean the previous gradient.
        optimizer.zero_grad()
        loss.backward()

        # Update loss for the batch.
        total_loss += loss.item()

        optimizer.step()

    print('[INFO] Epoch: {}/{}, Loss: {}'.format(epoch, epochs, total_loss/len(train_dataloader)))

    # Testing.
    model = model.eval()

    weighted_acc: float = 0
    total_samples: int = 0

    for _, (imgs, labels) in enumerate(iterable=test_dataloader, start=0):

        pred = torch.argmax(
            F.softmax(model(imgs), dim=1), dim=1
        ).view(-1)
        weighted_acc += torch.sum((pred==labels).float())
        total_samples += imgs.shape[0]

    print('[TESTING] Test accuracy: {}'.format(weighted_acc/total_samples))

[TRAINING] Training batch: 0/1563
[TRAINING] Training batch: 500/1563
[TRAINING] Training batch: 1000/1563
[TRAINING] Training batch: 1500/1563
[TRAINING] Training batch: 1562/1563
[INFO] Epoch: 0/2, Loss: 52.736910710789346
[TESTING] Test accuracy: 0.45809999108314514
[TRAINING] Training batch: 0/1563
[TRAINING] Training batch: 500/1563
[TRAINING] Training batch: 1000/1563
[TRAINING] Training batch: 1500/1563
[TRAINING] Training batch: 1562/1563
[INFO] Epoch: 1/2, Loss: 49.81868948054787
[TESTING] Test accuracy: 0.43939998745918274
